In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pretrained T5 model and tokenizer
model_path = "./fine_tuned_bug_fix_model/"  # Replace with your pretrained model directory
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

2024-12-23 23:20:26.980939: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
from datasets import Dataset

# Load dataset
df = pd.read_csv("test.csv") 
df = df[['buggy', 'fixed']]

In [4]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [14]:
input_texts = data['buggy'].tolist()
target_texts = data['fixed'].tolist()

In [17]:
import torch
# Prepare the evaluation function
def evaluate_model(input_texts, target_texts, tokenizer, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    predictions = []
    references = []

    for input_text, target_text in zip(input_texts, target_texts):
        # Tokenize the input text
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
        
        # Generate predictions
        outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
        
        # Decode the outputs
        decoded_pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        predictions.append(decoded_pred)
        references.append(target_text)
    
    return predictions, references

In [18]:
predictions, references = evaluate_model(input_texts, target_texts, tokenizer, model)

In [26]:
# Calculate metrics (Accuracy and CodeBLEU)
from sklearn.metrics import accuracy_score

# Example for simple accuracy (exact match between prediction and reference)
accuracy = accuracy_score(input_texts, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# CodeBLEU
from codebleu.score import compute_codebleu

codebleu_score = compute_codebleu(predictions, references)
print(f"CodeBLEU Score: {codebleu_score * 100:.2f}%")

Accuracy: 81.72%
CodeBLEU Score: 76.31%
